In [1]:
import numpy as np
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import collections
from tqdm import tqdm_notebook as tqdm
from datetime import datetime

from math import ceil, floor
import cv2
import json

import tensorflow as tf
import keras

import sys

#from keras_applications.mobilenet import MobileNet, preprocess_input
from keras_applications.inception_v3 import InceptionV3, preprocess_input

from sklearn.model_selection import ShuffleSplit
from keras.backend.tensorflow_backend import set_session
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Input
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

Using TensorFlow backend.


In [2]:
def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(dcm, window_center, window_width):

    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)

    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)

    return img

def bsb_window(dcm):
    brain_img = window_image(dcm, 40, 80)
    subdural_img = window_image(dcm, 80, 200)
    soft_img = window_image(dcm, 40, 380)

    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = (soft_img - (-150)) / 380
    bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)
    return bsb_img*255

def process_img(img):
    img=img.astype(np.uint8)
    img_grsc=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    dst = cv2.GaussianBlur(img_grsc,(5,5),cv2.BORDER_DEFAULT)
    ret, thresh = cv2.threshold(dst, 10, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours)>0:
        c = max(contours, key = cv2.contourArea)
        mask = np.zeros_like(img_grsc)
        cv2.drawContours(mask, [c], 0, 255, -1)
        out_img=cv2.bitwise_and(img,img,mask=mask)
        x,y,w,h=cv2.boundingRect(c)
        out_img=out_img[y:y+h,x:x+w]
        out_img=cv2.resize(out_img,(512,512))
        return out_img
    else:
        return img
        
def _read(path, desired_size=(512, 512)):

    dcm = pydicom.dcmread(path)

    

    try:
        # dcm.pixel_array might be corrupt (one case so far)
        img=process_img(bsb_window(dcm))/255
    except:
        img = np.zeros((desired_size[0],desired_size[1],3))        

    
    if desired_size != (512, 512):
        # resize image
        img = cv2.resize(img, desired_size, interpolation=cv2.INTER_LINEAR)

    return img

In [3]:
class TrainDataGenerator(keras.utils.Sequence):

    def __init__(self, list_IDs, labels, batch_size=1, img_size=(512, 512), 
                 img_dir='ds/stage_1_train_images/', *args, **kwargs):

        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indices]
        X, Y = self.__data_generation(list_IDs_temp)
        return X, Y

    def on_epoch_end(self):
        self.indices = np.arange(len(self.list_IDs))
        np.random.shuffle(self.indices)

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.img_size, 3))
        Y = np.empty((self.batch_size, 6), dtype=np.float32)
        
        for i, ID in enumerate(list_IDs_temp):
            X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
            Y[i,] = self.labels.loc[ID].values
        
        return X, Y
    
    
class TestDataGenerator(keras.utils.Sequence):

    def __init__(self, list_IDs, labels, batch_size=1, img_size=(512, 512), 
                 img_dir='ds/stage_1_test_images/', *args, **kwargs):

        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indices]
        X = self.__data_generation(list_IDs_temp)
        return X

    def on_epoch_end(self):
        self.indices = np.arange(len(self.list_IDs))

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.img_size, 3))
        
        for i, ID in enumerate(list_IDs_temp):
            X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
        
        return X

In [4]:
from keras.losses import binary_crossentropy
import keras.backend as K
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

In [12]:
def _initial_layer(input_dims):
    inputs = keras.layers.Input(input_dims)
    
    x = keras.layers.Conv2D(filters=3, kernel_size=(1, 1), strides=(1, 1), name="initial_conv2d")(inputs)
    x = keras.layers.BatchNormalization(axis=3, epsilon=1.001e-5, name='initial_bn')(x)
    x = keras.layers.Activation('relu', name='initial_relu')(x)
    
    return keras.models.Model(inputs, x)

class MyDeepModel:
    
    def __init__(self, engine, input_dims, batch_size=5, learning_rate=1e-3, 
                 decay_rate=1.0, decay_steps=1, weights="imagenet", verbose=1):
        
        self.engine = engine
        self.input_dims = input_dims
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.decay_rate = decay_rate
        self.decay_steps = decay_steps
        self.weights = weights
        self.verbose = verbose
        self._build()

    def weighted_log_loss(self,y_true, y_pred):
        """
        Can be used as the loss function in model.compile()
        ---------------------------------------------------
        """

        class_weights = np.array([2., 1., 1., 1., 1., 1.])

        eps = K.epsilon()

        y_pred = K.clip(y_pred, eps, 1.0-eps)

        out = -( y_true*K.log(y_pred)*class_weights+ (1.0 - y_true) * K.log(1.0 - y_pred) * class_weights)

        return K.mean(out, axis=-1)
    def _build(self):
        
        engine = self.engine(include_top=False, weights=self.weights, input_shape=(*self.input_dims, 3),
                             backend = keras.backend, layers = keras.layers,
                             models = keras.models, utils = keras.utils)

        x = engine.output

        x = keras.layers.GlobalAveragePooling2D(name='avg_pool_grb')(x)
        
        x = keras.layers.Dropout(0.25)(x)

        out = keras.layers.Dense(6, activation="sigmoid", name='dense_output')(x)

        self.model = keras.models.Model(inputs=engine.input, outputs=out)

        self.model.compile(loss="binary_crossentropy",optimizer=keras.optimizers.Adam())
        #print(self.model.summary())
    
    def fit(self, df, train_idx, img_dir, global_epoch):
        self.model.fit_generator(
            TrainDataGenerator(
                df.iloc[train_idx].index, 
                df.iloc[train_idx], 
                self.batch_size, 
                self.input_dims, 
                img_dir
            ),
            verbose=self.verbose,
            use_multiprocessing=True,
            workers=6,
            callbacks=[
                keras.callbacks.LearningRateScheduler(
                    lambda epoch: self.learning_rate * pow(self.decay_rate, floor(global_epoch / self.decay_steps))
                )
            ]
        )
    
    def predict(self, df, test_idx, img_dir):
        predictions = \
          self.model.predict_generator(
            TestDataGenerator(
                df.iloc[test_idx].index, 
                None, 
                self.batch_size, 
                self.input_dims, 
                img_dir
            ),
            verbose=1,
            use_multiprocessing=True,
            workers=4
        )

        return predictions[:df.iloc[test_idx].shape[0]]
    
    def save(self):
        model_json = self.model.to_json()
        with open("model_mobilenet_arch.json", "w") as json_file:
            json_file.write(model_json)
        self.model.save_weights("model_mobilenet_weights.h5")
    
    def load(self, path):
        self.model.load_weights(path)

In [13]:
def read_testset(filename="ds/stage_1_sample_submission.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    
    return df

def read_trainset(filename="ds/stage_1_train.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    
    duplicates_to_remove = [
        1598538, 1598539, 1598540, 1598541, 1598542, 1598543,
        312468,  312469,  312470,  312471,  312472,  312473,
        2708700, 2708701, 2708702, 2708703, 2708704, 2708705,
        3032994, 3032995, 3032996, 3032997, 3032998, 3032999
    ]
    
    df = df.drop(index=duplicates_to_remove)
    df = df.reset_index(drop=True)
    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    
    return df

    
test_df = read_testset()
df = read_trainset()

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from sklearn.metrics import log_loss
_TEST_IMAGES = 'ds/stage_1_test_images/'
_TRAIN_IMAGES = 'ds/stage_1_train_images/'

def run(model, df, train_idx, valid_idx, test_df, epochs):
    
    valid_predictions = []
    test_predictions = []
    for global_epoch in range(epochs):
        print(global_epoch)
        model.fit(df, train_idx, _TRAIN_IMAGES, global_epoch)
        
        #if global_epoch==epochs-1:
        test_predictions.append(model.predict(test_df, range(test_df.shape[0]), _TEST_IMAGES))
        valid_predictions.append(model.predict(df, valid_idx, _TRAIN_IMAGES))
        
        print("validation loss: %.4f" %
              weighted_loss_metric(df.iloc[valid_idx].values, np.average(valid_predictions, 
                                                                         axis=0, 
                                                                         weights=[2**i for i in range(len(valid_predictions))])
                                  )
             )
    model.save()
    return test_predictions, valid_predictions

# this is probably not correct, but works OK. 
#   Feel free to give feedback.
def weighted_loss_metric(trues, preds, weights=[0.2, 0.1, 0.1, 0.1, 0.1, 0.1], clip_value=1e-7):
    preds = np.clip(preds, clip_value, 1-clip_value)
    loss_subtypes = trues * np.log(preds) + (1 - trues) * np.log(1 - preds)
    loss_weighted = np.average(loss_subtypes, axis=1, weights=weights)
    return - loss_weighted.mean()

def wtlm(trues,preds):
    return log_loss(trues,preds,sample_weight=[2,1,1,1,1,1]*trues.shape[0])
# train set (90%) setand validation set (10%)
ss = ShuffleSplit(n_splits=5, test_size=0.1, random_state=42).split(df.index)

# will just do one fold
train_idx, valid_idx = next(ss)

# obtain model
model = MyDeepModel(engine=InceptionV3, input_dims=(299,299), batch_size=32, learning_rate=1e-3, 
                    decay_rate=0.75, decay_steps=1, weights="imagenet", verbose=1)

# run 2 epochs and obtain test + validation predictions
test_preds, _ = run(model, df, train_idx, valid_idx, test_df, 5)

0
Epoch 1/1
10879/18964 [================>.............] - ETA: 1:25:14 - loss: 0.1060

In [ ]:
len(test_preds)

In [ ]:
import pickle
#with open("saved_pkl.pkl","wb") as f:
#    pickle.dump(obj=test_preds,file=f)

In [ ]:
test_df.iloc[:, :] = np.average(test_preds, axis=0, weights=[2**i for i in range(len(test_preds))])

test_df = test_df.stack().reset_index()

test_df.insert(loc=0, column='ID', value=test_df['Image'].astype(str) + "_" + test_df['Diagnosis'])

test_df = test_df.drop(["Image", "Diagnosis"], axis=1)

test_df.head(20)

In [ ]:
test_df.to_csv('submission_three_ch_inception_v3_ep3.csv', index=False)

In [ ]:
%pip install vis